# Email-handling

Send automated emails to the participants of the sports week.

In [ ]:
import helper_functions as hf
from helper_functions.classes.player import Player
from helper_functions.setup.email_handling import get_email_address
import markdown
from pathlib import Path
import pandas as pd
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from bs4 import BeautifulSoup
from email import encoders
from typing import Callable, Literal
import yaml

hf.LOGGER.setLevel("INFO")
hf.LOGGER.info("Starting the script")


[sports_week: INFO] - Starting the script


In [ ]:
FPATH_SENT_MAILS = hf.FpathRegistry.get_path_hidden().parent.joinpath("mails_sent.yml")
if not FPATH_SENT_MAILS.exists() or yaml.safe_load(FPATH_SENT_MAILS.read_text(encoding="utf-8")) is None:
    FPATH_SENT_MAILS.parent.mkdir(parents=True, exist_ok=True)
    FPATH_SENT_MAILS.write_text(yaml.safe_dump({"fee_reminder": [], "schedule": [], "confirmation_reminder": []}), encoding="utf-8")

def get_sent_mails(key: Literal["fee_reminder", "schedule", "confirmation_reminder"]) -> list[str]:
    """Load the sent mails from the YAML file."""
    return yaml.safe_load(FPATH_SENT_MAILS.read_text(encoding="utf-8"))[key]

def update_sent_mails(key: Literal["fee_reminder", "schedule", "confirmation_reminder"], new_name: str):
    """Update the sent mails in the YAML file."""
    assert new_name in hf.DATA_NOW.nickname_to_name_df["nickname"].tolist(), f"{new_name} is not a valid nickname."
    sent_mails = yaml.safe_load(FPATH_SENT_MAILS.read_text(encoding="utf-8"))
    if new_name in sent_mails[key]:
        print(f"{new_name} already in {key}.")
        return
    sent_mails[key].append(new_name)
    FPATH_SENT_MAILS.write_text(yaml.safe_dump(sent_mails), encoding="utf-8")

# update_sent_mails("fee_reminder", "Pushy Bulldog")


## Sending out the emails

Following tutorial at https://realpython.com/python-send-email/ to send out mails programmatically.

In [53]:

def write_email_to_player(player_nick: str, 
                          player_email: str, 
                          email_text: str, 
                          email_subject: str,
                          sender_email: str, 
                          password: str, 
                          yaml_key: Literal["fee_reminder", "schedule", "confirmation_reminder"],
                          im_path: Path | None = None,
                          force=False):
    if not force and player_nick in get_sent_mails(yaml_key):
        hf.LOGGER.info(f"Mail to {player_nick} already sent. Skipping.")
        return

    html_text = markdown.markdown(email_text.replace("\\", "<br>"))
    soup = BeautifulSoup(html_text, 'html')
    plain_text = soup.get_text()
    # Turn these into plain/html MIMEText objects
    message = MIMEMultipart("alternative")
    message["Subject"] = email_subject
    message["From"] = sender_email
    message["To"] = player_email

    part1 = MIMEText(plain_text, "plain")
    part2 = MIMEText(html_text, "html")

    if im_path is not None:
        # Open png file in binary mode
        with open(im_path, "rb") as attachment:
            # Add file as application/octet-stream
            # Email client can usually download this automatically as attachment
            part3 = MIMEBase("application", "octet-stream")
            part3.set_payload(attachment.read())

        # Encode file in ASCII characters to send by email    
        encoders.encode_base64(part3)

        # Add header as key/value pair to attachment part
        part3.add_header(
            "Content-Disposition",
            f"attachment; filename= {im_path.name}",
        )
        message.attach(part3)

    # Add HTML/plain-text parts to MIMEMultipart message
    # The email client will try to render the last part first
    message.attach(part1)
    message.attach(part2)

    # - Actually send the mail
    hf.LOGGER.info(f"Sending mail to {player_nick}...")
    port = 587
    # Create secure connection with server and send email
    context = ssl.create_default_context()
    with smtplib.SMTP("ssmtp.mpe.mpg.de", port) as server:
        server.starttls(context=context) 
        server.login("fbalzer", password)
        server.sendmail(
            sender_email, [player_email, sender_email], message.as_string()
        )
    update_sent_mails(yaml_key, player_nick)
# for nickname in df.sort_values("response_timestamp")["nickname"]:
#     write_email_to_player(df.loc[nickname], sender_email, password, force=False)



In [41]:
# LOG IN

SENDER_MAIL = "@".join(["fbalzer", "mpe.mpg.de"])  # slight protection against sp am
SENDER_PASSWORD = input("Type your password and press enter:")



### Part 1: Reminder for fee payment

In [18]:
hidden_df = hf.FpathRegistry.get_hidden_responses()
unpaid = hidden_df[~hidden_df["has_paid_fee"]]
len(hidden_df), len(unpaid)


(185, 68)

In [43]:
def get_reminder_text(player: pd.Series) -> tuple[str, str]:
    base_text = hf.DATAPATH.joinpath("helper_texts/email_1_reminder_text.md").read_text(encoding="utf-8")
    first_name, nickname = player["name"].split()[0],  player["nickname"]
    text = base_text.format(first_name=first_name, nickname=nickname)
    email = player["email"]
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text



In [55]:
player = hidden_df.iloc[2]
player_email, email_text = get_reminder_text(player)
SUBJECT = "[SPORTS WEEK]: Fee Reminder"
write_email_to_player(player["nickname"], player_email, email_text, SUBJECT, SENDER_MAIL, SENDER_PASSWORD, "fee_reminder", force=False)


[sports_week: INFO] - Sending mail to Pushy Bulldog...


In [ ]:


for _, player in unpaid.iterrows():
    write_email_to_player(player, SENDER_MAIL, SENDER_PASSWORD, [], get_reminder_text, force=False)


In [ ]:


df = hf.get_players().set_index("nickname", drop=False).fillna("")
df.head()

df = df.join(hidden_df[[col for col in hidden_df.columns if not col in df.columns]], how="left")

addresses = []
df = df.fillna("")
for _, player in df.iterrows():
    if any([player[f"subteam_{sport}"] != "" for sport in hf.SPORTS_LIST]):
        addresses.append(get_email_address(player["email"]))

hf.copy_to_clipboard("; ".join(addresses))
# df.loc["Unimportant Beetle"]


In [2]:

def get_email_text_for_player(player: pd.Series) -> tuple[str, str, Path]:
    email_base_text = hf.DATAPATH.joinpath("helper_texts/email_blank_text.md").read_text(encoding="utf-8")
    first_name = player["name"].split()[0]
    nickname = player["nickname"]
    team_name = player["Team"]
    cloth_color = {"Team A": "dark", "Team B": "white", "Team C": "colorful"}[team_name]
    email = get_email_address(player["email"])
    signal_link = hf.DATAPATH.joinpath("hidden/signal_entry_link.txt").read_text(encoding="utf-8")
    
    player_obj = Player.from_series(player, list(hf.ALL_MATCHES))
    schedule = player_obj.get_schedule_for_mail()
    fpath = hf.DATAPATH.joinpath(f"assets/animal_pics/full_size/{player["nickname"].lower().replace(" ", "_")}.png")
    text = email_base_text.format(first_name=first_name, nickname=nickname, schedule=schedule, team_name=team_name, cloth_color=cloth_color, signal_link=signal_link)
    html_text = markdown.markdown(text.replace("\\", "<br>"))
    return email, html_text, fpath

# player = df.loc["Thankful Kakapo"]
# address, text, fpath = get_email_text_for_player(player)
email, text, fpath = get_email_text_for_player(df.loc["Elastic Pekingese"])
# print(email, "\n", fpath.name, "\nSports week (Apr 29 - May 3): Your schedule and information")
hf.copy_to_clipboard(text)
